In [33]:
#encoding=utf-8
import pandas as pd
import numpy as np

In [57]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/others/'
data = pd.read_csv(path + 'irrelevant_response_training_set.csv', encoding='utf8')
strategy_mat = pd.read_csv(path + 'irrelevant_response_strategy_matrix.csv', encoding='utf8')
model_list = ['Idclassfier','ifknowdebtor','causedebt','willingtopay','cutdebt','installation','setdueday']
data = data.rename(index=str, columns={'文本': 'text', '类别': 'label'})
strategy_mat = strategy_mat.rename(index=str, columns={'类别': 'label', '对策': 'strategy'})

print(data.shape)
print(strategy_mat.shape)
strategy_mat

(2054, 2)
(13, 8)


,label,Idclassfier,ifknowdebtor,ConfirmLoan,willingtopay,cutdebt,installment,strategy
0,讨价还价,1.0,1.0,0.0,1.0,0.0,0.0,cutdebt
1,讨价还价,1.0,1.0,1.0,1.0,0.0,0.0,施压
2,说出目的,0.0,0.0,1.0,1.0,1.0,1.0,ConfirmLoan
3,确认数额,1.0,1.0,0.0,1.0,1.0,1.0,willingtopay
4,请求重复,0.0,0.0,0.0,0.0,0.0,0.0,重复
5,请求等下打来,0.0,0.0,0.0,0.0,0.0,0.0,另选时间打来
6,其它通讯方式,1.0,1.0,1.0,0.0,1.0,1.0,重复
7,模糊确认,1.0,1.0,0.0,0.0,0.0,0.0,施压
8,回问身份,1.0,1.0,0.0,0.0,0.0,0.0,施压
9,回问身份,0.0,0.0,1.0,1.0,1.0,1.0,重复


In [32]:
for each_model in model_list:
    each_data = data[data[each_model]==0].copy()

In [56]:
data[data['label']=='其它通讯方式']
# data[data['label']=='不愿配合']
# data[data['label']=='帮还钱']
# data[data['label']=='讨价还价']

,text,label
1569,你加我微信说吧,其它通讯方式
1570,加我微信聊吧,其它通讯方式
1571,加我微信,其它通讯方式


In [50]:
(data['label'].unique())

array(['讨价还价', '说出目的', '确认数额', '请求重复', '请求等下打来', '其它通讯方式', '模糊确认', '回问身份',
       '还款方式', '故意岔开话题', '不愿配合'], dtype=object)

In [54]:
data.drop('text',axis=1).drop_duplicates(['label'])

,label
0,讨价还价
489,说出目的
543,确认数额
567,请求重复
957,请求等下打来
1569,其它通讯方式
1572,模糊确认
1612,回问身份
1681,还款方式
1691,故意岔开话题


In [41]:
# data[data['Idclassfier']==0]

In [55]:
data.label.value_counts()

请求等下打来    612
讨价还价      489
请求重复      390
故意岔开话题    231
不愿配合      132
回问身份       69
说出目的       54
模糊确认       40
确认数额       24
还款方式       10
其它通讯方式      3
Name: label, dtype: int64